# Cats and Dogs Classification Using CNN

In [1]:
#Metadata

%load_ext watermark
%watermark -a "Vutsal Singhal"
%watermark -u -n -t -z
%watermark -p keras
%watermark -m

Vutsal Singhal
last updated: Tue Jun 20 2017 21:15:02 IST


Using TensorFlow backend.


keras 2.0.4
compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.8.0-54-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


## Import libraries

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import plot_model
from keras.models import Sequential
from keras.models import load_model
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.datasets import mnist
from keras import backend as K
from imutils import paths
import numpy as np
import argparse
import cPickle
import h5py
import sys
import cv2
import os

K.set_image_dim_ordering('tf')
np.random.seed(7)

def image_to_feature_vector(image, size=(28, 28)):
	return cv2.resize(image, size)

data   = []
labels = []

Using TensorFlow backend.


## Image pre-processing

In [10]:
print("[INFO] pre-processing images...")
imagePaths = list(paths.list_images('../../../DATASETS/dogs_cats/train'))

for (i, imagePath) in enumerate(imagePaths):
	image    = cv2.imread(imagePath)
	label    = imagePath.split(os.path.sep)[-1].split(".")[0]
	features = image_to_feature_vector(image)
	data.append(features)
	labels.append(label)

	if i > 0 and i % 1000 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))

le     = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 2)
data   = np.array(data) / 255.0

print('[INFO] Saving processed images to disk...')
with h5py.File("cnn1_processed_images_and_labels.h5", "w") as hf:
	g = hf.create_group('data_labels')
	g.create_dataset('data', data=data)
	g.create_dataset('labels', data=labels)

[INFO] pre-processing images...
[INFO] processed 1000/25000
[INFO] processed 2000/25000
[INFO] processed 3000/25000
[INFO] processed 4000/25000
[INFO] processed 5000/25000
[INFO] processed 6000/25000
[INFO] processed 7000/25000
[INFO] processed 8000/25000
[INFO] processed 9000/25000
[INFO] processed 10000/25000
[INFO] processed 11000/25000
[INFO] processed 12000/25000
[INFO] processed 13000/25000
[INFO] processed 14000/25000
[INFO] processed 15000/25000
[INFO] processed 16000/25000
[INFO] processed 17000/25000
[INFO] processed 18000/25000
[INFO] processed 19000/25000
[INFO] processed 20000/25000
[INFO] processed 21000/25000
[INFO] processed 22000/25000
[INFO] processed 23000/25000
[INFO] processed 24000/25000
[INFO] Saving processed images to disk...


## Load pre-processed images and labels from disk

In [3]:
print('[INFO] Loding processed images and labels from disk...')
with h5py.File('cnn1_processed_images_and_labels.h5','r') as hf:
    g      = hf.get('data_labels')
    data   = np.array(g.get('data'))
    labels = np.array(g.get('labels'))

[INFO] Loding processed images and labels from disk...


## Creating train/test spilts

In [8]:
print("[INFO] constructing training/testing split...")
(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test  = np.array(X_test)
y_test  = np.array(y_test)
X_train = X_train.reshape(X_train.shape[0], 3, 28, 28).astype('float32')
X_test  = X_test.reshape(X_test.shape[0], 3, 28, 28).astype('float32')
num_classes = y_test.shape[1]

[INFO] constructing training/testing split...


## Loading saved models and weights from disk

In [11]:
print('[INFO] Loding saved model from disk...')
model = load_model('cnn1_dogs_cats_model.h5')
print('[INFO] Loding saved weights from disk...')
model.load_weights('cnn1_dogs_cats_weights.h5')

[INFO] Loding saved model from disk...
[INFO] Loding saved weights from disk...


## Defining model architecture

In [10]:
model = Sequential()
model.add(Conv2D(32, padding='valid', kernel_size=(3, 3), data_format='channels_first', input_shape=(3, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

## Traning model

In [8]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=50, shuffle=True, verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 15s - loss: 0.0476 - acc: 0.9820 - val_loss: 1.3515 - val_acc: 0.6788
Epoch 2/50
20000/20000 [==============================] - 15s - loss: 0.0505 - acc: 0.9811 - val_loss: 1.4308 - val_acc: 0.6780
Epoch 3/50
20000/20000 [==============================] - 14s - loss: 0.0570 - acc: 0.9790 - val_loss: 1.3515 - val_acc: 0.6866
Epoch 4/50
20000/20000 [==============================] - 15s - loss: 0.0461 - acc: 0.9822 - val_loss: 1.3975 - val_acc: 0.6768
Epoch 5/50
20000/20000 [==============================] - 13s - loss: 0.0508 - acc: 0.9810 - val_loss: 1.3827 - val_acc: 0.6772
Epoch 6/50
20000/20000 [==============================] - 15s - loss: 0.0513 - acc: 0.9808 - val_loss: 1.2618 - val_acc: 0.6720
Epoch 7/50
20000/20000 [==============================] - 16s - loss: 0.0522 - acc: 0.9807 - val_loss: 1.3293 - val_acc: 0.6752
Epoch 8/50
20000/20000 [==============================]

## Evaluating model

In [12]:
print('[INFO] Evaluating the model on test data...')
scores = model.evaluate(X_test, y_test, batch_size=100, verbose=1)
print("\nAccuracy: %.4f%%\n"%(scores[1]*100))

[INFO] Evaluating the model on test data...
4700/5000 [===========================>..] - ETA: 0s
Accuracy: 68.2000%



## Saving model and weights to disk

In [10]:
print('[INFO] Saving compiled model and weights to disk...')
model.save('cnn1_dogs_cats_model.h5')
model.save_weights('cnn1_dogs_cats_weights.h5')

[INFO] Saving compiled model and weights to disk...


## Plotting model

In [11]:
print('[INFO] Plotting model...')
plot_model(model, to_file='cnn1_keras_model.png', show_shapes=True)

[INFO] Plotting model...


## Testing model on unknown data

In [14]:
print('[INFO] Predicting images...')
cont = 'y'
while(True):
    if cont == 'n':
        print('Exit initiated...')
        cv2.destroyAllWindows()
        break

    inp = int(raw_input('Enter 0 < number < 12500: '))
    if inp >= 0 and inp < 12501:
        path  = '../../../DATASETS/dogs_cats/test/%s.jpg'%(inp)
        image = cv2.imread(path)
        feature = image_to_feature_vector(image)
        feature = np.reshape(feature,(1,3,28,28))

        print("[INFO] predicting new image: %s")%(path)
        prediction = model.predict(feature, batch_size=1, verbose=1)[0]
        classes    = model.predict_classes(feature, verbose=1)
        print('Probablities: %s')%(prediction)
        print('Class: %s')%(classes)

        if prediction[0] == 1:
            animal = 'Predicted [cat]'
        else:
            animal = 'Predicted: [dog]'

        cv2.putText(image, animal, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        cv2.namedWindow('cat/dog', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('cat/dog', image)
        if cv2.waitKey(0):
            cv2.destroyAllWindows()
    else:
        print 'Number not within the range!'
    
    cont = str(raw_input('Continue:(y/n) '))

[INFO] Predicting images...
Enter 0 < number < 12500: 2351
[INFO] predicting new image: ../../../DATASETS/dogs_cats/test/2351.jpg
1/1 [==============================] - 0s
Probablities: [  7.53419013e-22   1.00000000e+00]
Class: [1]
Continue:(y/n) y
Enter 0 < number < 12500: 125000
Number not within the range!
Continue:(y/n) y
Enter 0 < number < 12500: 3468
[INFO] predicting new image: ../../../DATASETS/dogs_cats/test/3468.jpg
1/1 [==============================] - 0s
Probablities: [ 0.  1.]
Class: [1]
Continue:(y/n) n
Exit initiated...
